# SJ Little Hotelier Data Migration

This notebook creates a pipeline that transforms 356 of [Saint Jameshostel](https://www.saint-james-hostel.co.uk)'s reservations from an Excel bookings sheet into an xlsx file that can be bulk uploaded into the [Little Hotelier](https://www.littlehotelier.com) property management software platform. The main challenges included: 
- reassigning the check-in dates on current bookings that fell before the upload date (07/11/2023)
- splitting first and last names
- assigning room type names based  on existing room names
- correctly detecting and renaming the sources of reservation

## Raw spreadsheet data

Reads in the raw data from an Excel bookings sheet. 

In [1]:
# imports key libraries and reads in the data
import pandas as pd
import numpy as np

raw_data = pd.read_excel('C:/Users/HUAWEI/Desktop/SJ Bulk upload/raw_data.xlsx')
raw_data.head(8)

,CHECK -IN,CHECK OUT,NIGHTS,GUESTS+,ROOM,BED,NAME,BOOKING PRICE,PAID ON,Booking Type,KD CASH,KD CARD,NOTES:,STATUS,Unnamed: 14
0,2023-11-04,2023-11-06,2.0,1.0,A,A,Vitória Sant'Ana,120.29,2023-04-11 00:00:00,Booking vc,NaN,10.0,NaN,Paid,NaN
1,2023-11-03,2023-11-06,3.0,1.0,5,5B,Ragi Tekleweyni Gereme,88.11,2023-03-11 00:00:00,Booking VC,NaN,10.0,NaN,Paid,NaN
2,2023-11-03,2023-11-06,3.0,1.0,21,21E,Jesse Henshall,105.30,2023-03-10 00:00:00,booking vcd,10.0,NaN,NaN,Paid,booking ok
3,2023-11-03,2023-11-06,3.0,1.0,22,22G,Nilesh Lakhman,77.52,2023-03-11 00:00:00,Booking VC,NaN,10.0,NaN,Paid,NaN
4,2023-11-03,2023-11-06,3.0,1.0,23,23C,Rosinski Marcin,77.52,2023-03-11 00:00:00,Booking VC,NaN,NaN,NaN,Paid,NaN
5,2023-11-03,2023-11-06,3.0,1.0,36,36B,Guenter Ruckhofer,80.10,2023-03-11 00:00:00,Booking VC,NaN,10.0,NaN,Paid,booking ok
6,2023-11-04,2023-11-06,2.0,1.0,2,2H,Paz Sanchez Negrete,51.31,2023-04-10 00:00:00,HW,NaN,10.0,she extended from 6th to 11th same room and sh...,Paid,Booking ok
7,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Remove spaces

Removes the spaces between bookings that were used to distinguish between check-in dates.

In [2]:
# loops through the raw data removign extra spaces 
non_nulls = []
for i in range(len(raw_data)):
    if pd.isnull(raw_data.iloc[i,:1][0])==False:
        non_nulls.append(i)
    
raw_data = raw_data.iloc[non_nulls,:].reset_index(drop=True)
raw_data.head(8)

,CHECK -IN,CHECK OUT,NIGHTS,GUESTS+,ROOM,BED,NAME,BOOKING PRICE,PAID ON,Booking Type,KD CASH,KD CARD,NOTES:,STATUS,Unnamed: 14
0,2023-11-04,2023-11-06,2.0,1.0,A,A,Vitória Sant'Ana,120.29,2023-04-11 00:00:00,Booking vc,NaN,10.0,NaN,Paid,NaN
1,2023-11-03,2023-11-06,3.0,1.0,5,5B,Ragi Tekleweyni Gereme,88.11,2023-03-11 00:00:00,Booking VC,NaN,10.0,NaN,Paid,NaN
2,2023-11-03,2023-11-06,3.0,1.0,21,21E,Jesse Henshall,105.30,2023-03-10 00:00:00,booking vcd,10.0,NaN,NaN,Paid,booking ok
3,2023-11-03,2023-11-06,3.0,1.0,22,22G,Nilesh Lakhman,77.52,2023-03-11 00:00:00,Booking VC,NaN,10.0,NaN,Paid,NaN
4,2023-11-03,2023-11-06,3.0,1.0,23,23C,Rosinski Marcin,77.52,2023-03-11 00:00:00,Booking VC,NaN,NaN,NaN,Paid,NaN
5,2023-11-03,2023-11-06,3.0,1.0,36,36B,Guenter Ruckhofer,80.10,2023-03-11 00:00:00,Booking VC,NaN,10.0,NaN,Paid,booking ok
6,2023-11-04,2023-11-06,2.0,1.0,2,2H,Paz Sanchez Negrete,51.31,2023-04-10 00:00:00,HW,NaN,10.0,she extended from 6th to 11th same room and sh...,Paid,Booking ok
7,2023-10-29,2023-11-07,9.0,1.0,22,22G,PHILIP BRAILEY,337.00,10/23/2023,Walk in,NaN,NaN,NaN,Paid,booking ok


## Remove early check-outs 

Removes any bookings whose check-out date fell before the upload date of 07/11/2023.

In [3]:
# reasigns raw_data as an index of itself including only check-out dates greater than start_date
import datetime
from datetime import datetime

start_date = datetime(2023, 11, 7, 0, 0)

raw_data = raw_data.loc[raw_data['CHECK OUT'] > start_date,:].reset_index(drop=True)
raw_data.head(8)

,CHECK -IN,CHECK OUT,NIGHTS,GUESTS+,ROOM,BED,NAME,BOOKING PRICE,PAID ON,Booking Type,KD CASH,KD CARD,NOTES:,STATUS,Unnamed: 14
0,2023-11-01,2023-11-08,7.0,1.0,2,2F,Fie Agergaard Jørler,146.65,2023-01-11 00:00:00,HW,NaN,10.0,NaN,Paid,booking ok
1,2023-11-05,2023-11-08,3.0,3.0,6,6,Laura Convard,274.05,2023-05-11 00:00:00,Booking vcd,10.0,NaN,NaN,paid,booking ok
2,2023-11-05,2023-11-08,1.0,1.0,23,23H,Alan McLaughlan,51.64,2023-05-11 00:00:00,HW,NaN,10.0,NaN,paid,booking ok
3,2023-11-06,2023-11-08,2.0,1.0,36,36E,Andrea Manni,41.31,2023-06-11 00:00:00,HW,NaN,NaN,NaN,Paid,booking ok
4,2023-11-06,2023-11-08,2.0,1.0,5,5B,Brendda Lacerda,44.56,2023-06-11 00:00:00,Booking VC,10.0,NaN,NaN,Paid,NaN
5,2023-10-26,2023-11-09,14.0,1.0,22,22C,Mike Jacobs,466.20,10/26/2023,Booking VCD,NaN,NaN,Extension same bed,paid,booking ok
6,2023-11-01,2023-11-09,8.0,4.0,35,35all,Hanna OBrien,954.72,2023-01-11 00:00:00,hw,NaN,10.0,NaN,paid,booking ok
7,2023-11-02,2023-11-09,7.0,1.0,24,24B,Hisay Lama,197.82,10/31/2023,booking hcn,NaN,10.0,EXTENSION SAME BED,Paid,booking ok


## Reformat dates

Reformats datetime objects to text with the day/month/year format. 

In [4]:
# uses list comprehension to reasign all dates into strings with %d-%m-%Y format
check_in = [x.strftime("%d-%m-%Y") for x in raw_data['CHECK -IN']]
check_out = [x.strftime("%d-%m-%Y") for x in raw_data['CHECK OUT']]

pd.DataFrame({'Check-in' : check_in,
             'Check-out' : check_out}).head(8)

,Check-in,Check-out
0,01-11-2023,08-11-2023
1,05-11-2023,08-11-2023
2,05-11-2023,08-11-2023
3,06-11-2023,08-11-2023
4,06-11-2023,08-11-2023
5,26-10-2023,09-11-2023
6,01-11-2023,09-11-2023
7,02-11-2023,09-11-2023


## Reassign early check-in dates

Reassigns check-in dates on current bookings that fell before the upload date of 07/11/2023 as the Little Hotelier bulk update tool does not allow the for the input of bookings with check-in dates in the past. 

In [5]:
# uses an if else statement to reasign check-in dates less than start_date to start_date
adjusted_check_ins = []
for i in range(len(check_in)):
    current = datetime.strptime(check_in[i], '%d-%m-%Y')
    if current < start_date:
        adjusted_check_ins.append('07-11-2023')
    else:
        adjusted_check_ins.append(check_in[i])
        
adjusted_check_ins[0:9]

['07-11-2023',
 '07-11-2023',
 '07-11-2023',
 '07-11-2023',
 '07-11-2023',
 '07-11-2023',
 '07-11-2023',
 '07-11-2023',
 '07-11-2023']

## Split first / last name

Splits the first and last names by assuming the first word is the first name and anything that follows is part of the last name. Bookings with only one name are assumed to be the first name and the last name field is left blank.

### Get first names

In [6]:
# uses list comprehension to split names by the first word 
first_names = [x.split()[0] for x in list(raw_data['NAME'])]
first_names[0:9]

['Fie',
 'Laura',
 'Alan',
 'Andrea',
 'Brendda',
 'Mike',
 'Hanna',
 'Hisay',
 'Amanda']

### Get last names

In [7]:
# uses list comprehension to split names by the first word
# all words thereafter are joined 
# if the name contains only 1 word 'NA' is inputted
last_names = [' '.join(x.split()[1:len(x.split())]) if len(x.split())>1 else 'NA' for x in list(raw_data['NAME'])]
last_names[0:9]

['Agergaard Jørler',
 'Convard',
 'McLaughlan',
 'Manni',
 'Lacerda',
 'Jacobs',
 'OBrien',
 'Lama',
 'Rudgeri']

## Assign room type names

Assigns room type names by grouping existing unique room names and assigning the correct room type name. 

### Define variables

In [8]:
# defines arrays of room types
eight_bed_mixed = [21, 22, 23, 24]
eight_bed_female = [2, 3, 5]
six_bed = [32, 36]
four_bed = [33, 34, 35]
triple = [1, 6, 20, 25]
private_shared = [30, 31, 37]
private_ensuite = ['A', 'B']
private_twin = 'C'
male_staff = 7
female_staff = 4 

# defines room type array names 
room_type_names = ['eight_bed_mixed',
                  'eight_bed_female',
                  'six_bed',
                  'four_bed',
                  'triple',
                  'private_shared',
                  'private_ensuite',
                  'private_twin',
                  'male_staff',
                  'female_staff']

# defines new room type names
new_room_type_names = ['1---8 Bed Dorm Mixed',
                      '2 --- 8 Bed Dorm Female',
                      '3 --- 6 Bed Dorm Mixed',
                      '4 --- 4 Bed Dorm Mixed',
                      '5 --- Triple Bunk (private)',
                      '6 --- Private Double Shared Bathroom',
                      '7 --- Private Double En Suite',
                      '8 --- Private Twin En Suite',
                      '9 --- Male Staff Dorm',
                      '9F --- Female Staff Dorm']

### Loop through rooms

In [9]:
# checks which room type a room fits into
def is_in_room(room):
    for i in range(len(room_type_names)):
        try:
            if (room in eval(room_type_names[i])):
                return new_room_type_names[i]
        except:
            i = i
            
        try:
            if (room == eval(room_type_names[i])):
                return new_room_type_names[i]
        except:
            i = i
            
    return 'NA'


# returns an list of multiple room types
def get_room_types(data):
    room_types = []
    for i in range(len(data)):
        room = data['ROOM'][i]
        room_types.append(is_in_room(room))
    return room_types


# stores room types
room_type = get_room_types(raw_data)
room_type[0:9]

['2 --- 8 Bed Dorm Female',
 '5 --- Triple Bunk (private)',
 '1---8 Bed Dorm Mixed',
 '3 --- 6 Bed Dorm Mixed',
 '2 --- 8 Bed Dorm Female',
 '1---8 Bed Dorm Mixed',
 '4 --- 4 Bed Dorm Mixed',
 '1---8 Bed Dorm Mixed',
 '2 --- 8 Bed Dorm Female']

## Detect and rename source of reservation

Detects reservation source by searching for unique strings in the 'Booking Type' field before renaming according to Little Hotelier specifications. 

### Define strings

In [10]:
# unique strings to detect the way reservation types are currently written
res_strings = ['booking',
              'hw',
              'group',
              'walk',
              'bb',
              'expedia',
              'trip']

# the way reservation types should now be written
res_correct = ['Booking.com',
               'Hostelworld Group',
               'Direct Booking',
               'Direct Booking',
               'Direct Booking',
               'Expedia',
               'Trip.com']

### Replace source of reservation

In [11]:
# imports re for string matching
import re

# returns the correct source of reservation string for a given booking type
def which_booking_type(booking_type):
    for i in range(len(res_strings)):
        if re.search(res_strings[i], booking_type):
            return res_correct[i]
    return 'NA'
            
# loops over the booking types and returns a list of source of reservation strings
res_sources = []
for i in range(len(raw_data)):
    booking_type = raw_data['Booking Type'][i].lower()
    res_sources.append(which_booking_type(booking_type))
    
res_sources[0:9]

['Hostelworld Group',
 'Booking.com',
 'Hostelworld Group',
 'Hostelworld Group',
 'Booking.com',
 'Booking.com',
 'Hostelworld Group',
 'Booking.com',
 'Direct Booking']

## Create new dataframe

Creates a data dataframe of all transformed fields and adds payment information to the notes column. 

### Define other columns

In [12]:
# other columns types requiring no transformation or being filled with blanks
res_source = raw_data['Booking Type']
res_id = ['' for x in range(len(raw_data))]
adults = raw_data['GUESTS+']
children = ['' for x in range(len(raw_data))]
infants = ['' for x in range(len(raw_data))]
room_number = raw_data['BED']
room_total = raw_data['BOOKING PRICE']
notes = raw_data['STATUS']

### Combine all columns

In [13]:
# creates a new dataframe with the correct mappings
new_df = pd.DataFrame({ 'First name': first_names,
             'Last name' : last_names,
             'Source of reservation' : res_sources,
             'Reservation ID' : res_id,
             'Adults' : adults,
             'Children' : children,
             'Infants' : infants,
             'Check-in' : adjusted_check_ins,
             'Check-out' : check_out,
             'Room type' : room_type,
             'Room number' : room_number,
             'Room total' : room_total,
             'Notes' : notes})
new_df.head(8)

,First name,Last name,Source of reservation,Reservation ID,Adults,Children,Infants,Check-in,Check-out,Room type,Room number,Room total,Notes
0,Fie,Agergaard Jørler,Hostelworld Group,,1.0,,,07-11-2023,08-11-2023,2 --- 8 Bed Dorm Female,2F,146.65,Paid
1,Laura,Convard,Booking.com,,3.0,,,07-11-2023,08-11-2023,5 --- Triple Bunk (private),6,274.05,paid
2,Alan,McLaughlan,Hostelworld Group,,1.0,,,07-11-2023,08-11-2023,1---8 Bed Dorm Mixed,23H,51.64,paid
3,Andrea,Manni,Hostelworld Group,,1.0,,,07-11-2023,08-11-2023,3 --- 6 Bed Dorm Mixed,36E,41.31,Paid
4,Brendda,Lacerda,Booking.com,,1.0,,,07-11-2023,08-11-2023,2 --- 8 Bed Dorm Female,5B,44.56,Paid
5,Mike,Jacobs,Booking.com,,1.0,,,07-11-2023,09-11-2023,1---8 Bed Dorm Mixed,22C,466.20,paid
6,Hanna,OBrien,Hostelworld Group,,4.0,,,07-11-2023,09-11-2023,4 --- 4 Bed Dorm Mixed,35all,954.72,paid
7,Hisay,Lama,Booking.com,,1.0,,,07-11-2023,09-11-2023,1---8 Bed Dorm Mixed,24B,197.82,Paid


## Save dataframe

Saves the dataframe as an xlsx after sorting by guest type. It is important to distinguish between single and multi-guest bookings as the latter cannot be automatically assigned extra beds by Little Hotelier and so must be adjusted manually. 

In [14]:
# sorts the bookings by guest number (ascending) and saves as xlsx
new_df = new_df.sort_values(by='Adults', ascending=True)
new_df.to_excel('transformed_df.xlsx', index=False)